In [1]:
import nbimporter
from dataclasses import dataclass, field, InitVar, asdict
from setup import *

In [2]:
# ersetzt z.Zt die endgültige entry Funktion
def entry(key):
    return 42.

In [3]:
# Definition der relevanten Spaltennamen für den Sektor T 

@dataclass
class TColVars:
    energy:float = None
    mileage:float = None
    transport_capacity_pkm:float = None
    transport_capacity_tkm:float = None
    demand_electricity:float = None   
    demand_epetrol:float = None
    demand_ejetfuel:float = None
    demand_ediesel:float = None
    demand_hydrogen:float = None
    CO2e_cb:float = None
    CO2e_total:float = None
    change_energy_MWh:float = None
    change_energy_pct:float = None
    change_CO2e_t:float = None
    change_CO2ee_pct:float = None
    CO2e_total_2021_estimated:float = None
    cost_climate_saved:float = None
    change_km:float = None
    action:str=None  # Todo: Prüfen, wie gehen wir mit den Maßnahmen in der Spalte action in Python um (?) 
    invest_pa:float = None
    invest_pa_com:float = None
    invest_pa_outside:float = None
    invest:float = None
    invest_com:float = None
    invest_outside:float = None
    cost_wage:float = None
    ratio_wage_to_emplo:float = None
    demand_emplo:float = None
    demand_emplo_com:float = None
    emplo_existing:float = None
    demand_emplo_new:float = None
    base_unit:float = None
    invest_per_x:float = None

        

In [6]:
# Definition der Zeilennamen für den Sektor T
@dataclass
class T3X:
    # Klassenvariablen für T   
    air_inter: TColVars = TColVars()
    air_dmstc: TColVars = TColVars() # ggf. löschen, da 0 in Szenario 
        
    car_it_ot: TColVars = TColVars()
    car_ab: TColVars = TColVars()
    bus: TColVars = TColVars()
   
    ldt_it_ot: TColVars = TColVars()
    ldt_ab: TColVars = TColVars()
    mhd_it_ot: TColVars = TColVars()
    mhd_ab: TColVars = TColVars()

    rail_ppl: TColVars = TColVars()
    metro: TColVars = TColVars()
    rail_gds: TColVars = TColVars()
    
    ship_dmstc: TColVars = TColVars()
    ship_inter: TColVars = TColVars()
        
    foot: TColVars = TColVars()
    cycl: TColVars = TColVars()
        
    #übergeordnete Zeilen 
    T: TColVars = TColVars()
    planning: TColVars = TColVars()   
    air: TColVars = TColVars()
    street: TColVars = TColVars()
    rail: TColVars = TColVars()  
    shp: TColVars = TColVars()
    others: TColVars = TColVars()
    

    # Maßnahmen 
    action_car: TColVars = TColVars() 
    action_road_maintenance: TColVars = TColVars()
    action_charge_points: TColVars = TColVars()
    action_bus_infrastructure: TColVars = TColVars()
    action_bus_invest: TColVars = TColVars()
    action_mhd_engine_change: TColVars = TColVars()     
    action_rail_maintenance: TColVars = TColVars()
    action_rail_invest: TColVars = TColVars()
        
        
    #Bereitstellung (Energieträgersummen)
    s_epetrol: TColVars = TColVars()   # Todo: in Excel ist die Bennenung "S_petrol, s_diesel ..." , ggf. umbenennen 
    s_ejetfuel: TColVars = TColVars()
    s_ediesel: TColVars = TColVars()
    s_elec: TColVars = TColVars()
    s_hydrogen: TColVars = TColVars()



    # erzeuge dictionry
    def dict(self):
        return asdict(self)

In [9]:
# Berechnungsfunktion im Sektor T für 203X
# Parameter root: oberste Generator Instanz
def Transport203X_calc(root):
   try:
        
#NACHFRAGE:

#Zeile Verkehr 
        
        T=root.t3X.T

        #1    
        if entry("In_T_rt3")=="city":         
            T.transport_capacity_pkm=(
                entry("In_M_population_com_203X")
                * ass("Ass_T_D_ratio_trnsprt_ppl_to_ppl_city")
           )
                
        elif entry("In_T_rt3")=="smcty":
            T.transport_capacity_pkm= (
                entry("In_M_population_com_203X")
                *ass("Ass_T_D_ratio_trnsprt_ppl_to_ppl_smcity")
            )
        
        elif entry("In_T_rt3")=="rural":
            T.transport_capacity_pkm=(
                entry("In_M_population_com_203X")
                *ass("Ass_T_D_ratio_trnsprt_ppl_to_ppl_rural")
            )
        else:
            T.transport_capacity_pkm=(
                entry("In_M_population_com_203X")
                *ass("Ass_T_D_trnsprt_ppl_nat")
                /entry("In_M_population_nat")
            )
    
        #result: 959.000.000.000 Pers km





#Zeile internationaler Flugverkehr (Güter und Personen):

        air_inter = root.t3X.air_inter # Todo: Prüfen "t3X" richtiger Objektname auf Generatorebene (?) 
        
        # 1
        air_inter.demand_ejetfuel = (
            ass("Ass_T_D_Air_nat_EB_2050")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
        )
        #result: 42.000.000  MWh/a
        
        #2 
        air_inter.energy= air_inter.demand_ejetfuel
        #result: 42.000.000  MWh/a
            
        
        #3
        air_inter.transport_capacity_pkm=(
            root.t18.air_inter.transport_capacity_pkm
            /root.t18.air_inter.demand_jetfuel
            *air_inter.demand_ejetfuel
        )
        #result: 22.787.103.078 Pers km
        
        #4
        air_inter.transport_capacity_tkm=(
            root.t18.air_inter.transport_capacity_pkm
            /root.t18.air_inter.demand_jetfuel
            *air_inter.demand_ejetfuel
        )
        #result:  556.841.827 t km
        
        #5
        air_inter.CO2e_cb = (
            air_inter.demand_ejetfuel
            *ass('Ass_T_S_jetfuel_EmFa_tank_wheel_2050')
        )
        #result: 11.076.307  t/a
        
        #6 
        air_inter.change_energy_MWh=(
            air_inter.energy
            -root.t18.air_inter.energy
        )
        #result -71.722.222 MWh
        
        #7 
        air_inter.change_energy_pct=(
            air_inter.change_energy_MWh
            /root.t18.air_inter.energy
        )
        #result: -63%

        #8
        air_inter.change_CO2e_t=(
            air_inter.CO2e_cb
            -root.t18.air_inter.CO2e_cb
        )
        #result: -18.943.985  t/a

        #9
        air_inter.change_CO2ee_pct=(
            air_inter.change_CO2e_t
            /root.t18.air_inter.CO2e_cb
        )
        #result: -63,10%
        
        #10
        air_inter.CO2e_total_2021_estimated=(
            root.t18.air_inter.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 28.024.879 t/a

        #11
        air_inter.cost_climate_saved(
            (air_inter.CO2e_total_2021_estimated-air_inter.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 80.971.802.574 €

        # 12 
        air_inter.change_km=(
             air_inter.transport_capacity_pkm
            -root.t18.air_inter.transport_capacity_pkm
        )
        #Todo: Es wird nur die Änderung der Pkm berechnet, Änderung der Tonnen-km ggf. hinzufügen?
        # result: -38.912.896.922 Pers km
        
        # 13 
        air_inter.action="Umstellung auf e-Kerosin sowie Effizienzsteigerung in der Antriebstechnologie"
        # Todo: Prüfen, ob Berücksichtigung Maßnahmen so passt
        # Todo: ggf. Maßnahme mit Kosten belegen 


#Zeile nationaler Flugverkehr (Güter und Personen) # Todo: Prüfen ob hier Werte benötigt werden, um Differenz 2018 zu 203X zu berechnen?, ggf. auch obsolet?
       
        air_dmstc  = root.t3X.air_dmstc
        
        # 1
        air_dmstc.demand_ejetfuel = 0
        #result 0 MWh
        
        
        #2
        air_dmstc.energy = air_dmstc.demand_ejetfuel
        #result: 0 MWh
        
        #3
        air_dmstc.transport_capacity_pkm= 0
        
        #4
        air_dmstc.CO2e_cb = 0
            
        #5 
        air_dmstc.change_energy_MWh=(
             air_dmstc.energy
            -root.t18.air_dmstc.energy
        )
        #result -7.913.611 MWh

        
        #6 
        air_dmstc.change_energy_pct=(
             air_dmstc.change_energy_MWh
            /root.t18.air_dmstc.energy
        )
        #result: -100 %

        #7
        air_dmstc.change_CO2e_t=(
             air_dmstc.CO2e_cb
            -root.t18.air_dmstc.CO2e_cb
        )
        #result: -2.085.724  t/a


        #8
        air_dmstc.change_CO2ee_pct=(
            air_dmstc.change_CO2e_t
            /root.t18.air_dmstc.CO2e_cb
        )
        #result: -100,00%

        
        #9
        air_dmstc.CO2e_total_2021_estimated=(
            root.t18.air_dmstc.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 1.947.088 t/a


        #10
        air_dmstc.cost_climate_saved(
            (air_dmstc.CO2e_total_2021_estimated-air_dmstc.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 9.302.212.965 €

        #11 
        air_dmstc.change_km=(
            air_dmstc.transport_capacity_pkm-
            -root.t18.air_dmstc.transport_capacity_pkm
        )
        # result: -10.100.000.000 Pers km
        
        #12 
        air_dmstc.action= "Verlagerung innerdeutscher Flüge auf den Schienenverkehr"
         # Kosten für diese Maßnahme werden im Schienenverkehr berücksichtigt 

            
# Zeile CRF 1.A.3.a - nationaler Luftverkehr plus internationaler Luftverkehr  

        air = root.t3X.air  
        
        # 1
        air.demand_ejetfuel=(
            air_inter.demand_ejetfuel
            +air_dmstc.demand_ejetfuel
        )
        #result: 42.000.000  MWh/a
        
        #2 
        air.energy=(
            air_inter.energy
            +air_dmstc.energy
        )
        #result: 42.000.000  MWh/a
            
        
        #3
        air.CO2e_cb=(
            air_inter.CO2e_cb
            +air_dmstc.CO2e_cb
        )
        #result: 11.076.307  t/a
        
        #4 
        air.change_energy=(
            air.energy
            -root.t18.air.energy
        )
        #result: -79.635.833 MWh
        
        #5 
        air.change_energy_pct=(
            air.change_energy_MWh
            /root.t18.air.energy
        )
        #result: -65%
        # Todo Zeile Air in 2018 fehlt noch 

        #6
        air.change_CO2e_t=(
            air.CO2e_cb
            -root.t18.air.CO2e_cb
        )
        #result: -21.029.709  t/a

        #7
        air.change_CO2ee_pct=(
            air.change_CO2e_t
            /root.t18.air.CO2e_cb
        )
        #result: -65,50%
        # Todo Zeile Air in 2018 fehlt noch 
            
        #8
        air.CO2e_total_2021_estimated=(
            air_inter.CO2e_total_2021_estimated
            +air_dmstc.CO2e_total_2021_estimated
        )
        #result: 29.971.967 t/a


        #9
        air.cost_climate_saved=(
            air_inter.CO2e_total_2021_estimated
            +air_dmstc.CO2e_total_2021_estimated
        )
        # result: 90.274.015.539 €

        # 10
        air.change_km=(
            air_inter.change_km
            +air_dmstc.change_km
        )
        # result: -49.012.896.922,33 €/a


          
 #Zeile MIV innerorts / außerorts 
        car_it_ot=root.t3X.car_it_ot

       #1
        if entry("In_T_rt3")=="city": 
            car_it_ot.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_it_ot.mileage     # Todo: Bezeichnung prüfen  t18 oder t 
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage
                 )
                *(ass("Ass_T_D_trnsprt_ppl_city_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car4_frac_2050")
                 )
            )
        elif entry("In_T_rt3")=="smcty": 
             car_it_ot.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_it_ot.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage
                 )
                * (ass("Ass_T_D_trnsprt_ppl_smcty_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car4_frac_2050")
                  )
             )
                
        elif entry("In_T_rt3")=="rural":
            car_it_ot.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_it_ot.mileage
                /(root.t18.car_it_ot.mileage+root.t18.car_ab.mileage)
                *(ass("Ass_T_D_trnsprt_ppl_rural_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car4_frac_2050")
                 )
            )
            
        else:
            car_it_ot.transport_capacity_pkm = (
                root.T.transport_capacity_pkm
                *root.t18.car_it_ot.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage)
                *(ass("Ass_T_D_trnsprt_ppl_nat_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car4_frac_2050")
                 )
            )
        # result: 372.824.012.822 Pers km
                
        #2
        car_it_ot.mileage=(
             car_it_ot.transport_capacity_pkm
             /ass("Ass_T_D_lf_ppl_Car_2050")
         )
         #result: 155.343.338.676 Fz km
       
        #3
        car_it_ot.demand_electricity=(
            car_it_ot.mileage
            *ass("Ass_T_S_Car_frac_bev_with_phev_mlg_2050")
            *ass("Ass_T_S_Car_SEC_elec_it_at_2030")
        )
        # result: 23.057.648  MWh/a
        
        #4 
        car_it_ot.demand_epetrol=(
            car_it_ot.mileage
            *ass("Ass_T_S_Car_frac_petrol_with_phev_mlg_2050")
            *ass("Ass_T_S_Car_SEC_petrol_it_at_2050")
        )
        # result: 4.486.626  MWh/a
        
        #5 
        car_it_ot.energy=(
            car_it_ot.demand_electricity
            +car_it_ot.demand_epetrol
        )
        # result: 27.544.274 MWh
        
        
        #6
        car_it_ot.CO2e_cb=(
            car_it_ot.demand_epetrol
            *ass("Ass_T_S_petrol_EmFa_tank_wheel_2050")  
        )
        
        #result: 1.216.008  t/a
        
        
        #7 
        car_it_ot.change_energy_MWh=(
             car_it_ot.energy
            -root.t18.car_it_ot.energy
        )
        #result -269.303.528 MWh


        
        #8
        car_it_ot.change_energy_pct=(
             car_it_ot.change_energy_MWh
            /root.t18.car_it_ot.energy
        )
        #result: -91 %

        #9
        car_it_ot.change_CO2e_t=(
             car_it_ot.CO2e_cb
            -root.t18.car_it_ot.CO2e_cb
        )
        #result: -74.516.133  t/a



        #10
        car_it_ot.change_CO2ee_pct=(
            car_it_ot.change_CO2e_t
            /root.t18.car_it_ot.CO2e_cb
        )
        #result: -98,39%


        
        #11
        car_it_ot.CO2e_total_2021_estimated=(
            root.t18.car_it_ot.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 70.698.315 t/a


        #12
        car_it_ot.cost_climate_saved(
            (car_it_ot.CO2e_total_2021_estimated-car_it_ot.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 331.951.720.779 € 
        
        #13
        car_it_ot.change_km=(
            car_it_ot.transport_capacity_pkm-root.t18.car_it_ot.transport_capacity_pkm
        )
        # -284.880.198.685 Pers km

        


 #Zeile MIV Autobahn 
        car_ab=root.t3X.car_ab

       #1
        if entry("In_T_rt3")=="city": 
            car_ab.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_ab.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage
                 )
                *(ass("Ass_T_D_trnsprt_ppl_city_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_city_car4_frac_2050")
                 )
            )
        elif entry("In_T_rt3")=="smcty": 
             car_ab.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_ab.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage
                 )
                * (ass("Ass_T_D_trnsprt_ppl_smcty_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_smcty_car4_frac_2050")
                  )
             )
                
        elif entry("In_T_rt3")=="rural":
            car_it_ot.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_ab.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage
                 )
                *(ass("Ass_T_D_trnsprt_ppl_rural_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_rural_car4_frac_2050")
                 )
            )
            
        else:
            car_ab.transport_capacity_pkm = (
                T.transport_capacity_pkm
                *root.t18.car_it_ab.mileage
                /(root.t18.car_it_ot.mileage
                  +root.t18.car_ab.mileage)
                *(ass("Ass_T_D_trnsprt_ppl_nat_car1_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car2_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car3_frac_2050")
                  +ass("Ass_T_D_trnsprt_ppl_nat_car4_frac_2050")
                 )
            )

        # result: 164.215.987.178 Pers km

                
        #2
        car_ab.mileage=(
             car_ab.transport_capacity_pkm
             /ass("Ass_T_D_lf_ppl_Car_2050")
         )
         #result: 68.423.327.991 Fz km

       
        #3
        car_ab.demand_electricity=(
            car_ab.mileage
            *ass("Ass_T_S_Car_frac_bev_with_phev_mlg_2050")
            *ass("Ass_T_S_Car_SEC_elec_ab_2030")
        )
        # result: 16.236.275  MWh/a

        
        #4 
        car_ab.demand_epetrol=(
            car_ab.mileage
            *ass("Ass_T_S_Car_frac_petrol_with_phev_mlg_2050")
            *ass("Ass_T_S_Car_SEC_petrol_ab_2050")
        )
        # result: 2.286.266  MWh/a

        #5
        car_ab.energy=(
            car_ab.demand_electricity
            +car_ab.demand_epetrol
        )
        # result: 18.522.541 MWh

        
        #6 
        car_ab.CO2e_cb=(
            car_ab.demand_epetrol
            *ass("Ass_T_S_petrol_EmFa_tank_wheel_2050")  
        )
        #result: 619.646  t/a
        
        #7 
        car_ab.change_energy_MWh=(
             car_ab.energy
            -root.t18.car_ab.energy
        )
        #result -130.595.454 MWh

        
        #8
        car_ab.change_energy_pct=(
             car_ab.change_energy_MWh
            /root.t18.car_ab.energy
        )
        #result: -88%

        #9
        car_ab.change_CO2e_t=(
             car_ab.CO2e_cb
            -root.t18.car_ab.CO2e_cb
        )
        #result: -37.428.743  t/a


        #10
        car_ab.change_CO2ee_pct=(
            car_ab.change_CO2e_t
            /root.t18.car_ab.CO2e_cb
        )
        #result: -98,37%

        
        #11
        car_ab.CO2e_total_2021_estimated=(
            root.t18.car_ab.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 35.519.357 t/a



        #12
        car_ab.cost_climate_saved(
            (car_ab.CO2e_total_2021_estimated-car_ab.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 166.733.370.771 €
        
        #13
        car_ab.change_km=(
            car_ab.transport_capacity_pkm-root.t18.car_ab.transport_capacity_pkm
        )
        # -125.479.801.315 Pers km

# Zeile "Kauf von E-Autos + Anzahl E-PKW im Zieljahr " action_car
            
        action_car=root.t3X.action_car
            
        #1 
        action_car.action="Private Investitionen in E-PKW"
            
        #2 
        action_car.base_unit=(
        (car_ab.transport_capacity_pkm
        +car_it_ot.transport_capacity_pkm)
            *Ass_S_ratio_bev_car_per_trnsprt_ppl_2050
            *Ass_S_bev_cars_frac_2050
        )  
        # result: 29.000.000 BEV 
            
        #3 
        action_car.invest_per_x=(fact("Fact_T_S_Car_average_price_2020"))
        # result: 26.039 €  
            
        #4 
        action_car.invest=(
            action_car.base_unit
            *action_car.invest_per_x
        )
        # result: 755.135.142.857 €
            
        #5 
        action_car.invest_com=(
            action_car.invest
            *ass("Ass_T_C_ratio_public_sector_0")
        )
        #result: 0 €
        
        #6
        action_car.invest_pa=(
            action_car.invest
            /entry("In_M_duration_target")
        )
        #result: 83.903.904.762 €/a
        
        #7
        action_car.invest_pa_com=(
            action_car.invest_com
            /entry("In_M_duration_target")
        )
        #result: 0 €   
            
            
 #Zeile Linienbusse (im ÖPNV Nahverkehr) 
 
        bus=root.t3X.bus

       #1
        if entry("In_T_rt3")=="city": 
            bus.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.bus.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_city_pt_frac_2050")
            )
    
    
    
        elif entry("In_T_rt3")=="smcty":
            bus.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.bus.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_smcty_pt_frac_2050")

            )            
    
                
        elif entry("In_T_rt3")=="rural":
            bus.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.bus.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_rural_pt_frac_2050")
            )   
            
        else:
            bus.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.bus.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_nat_pt_frac_2050")
                )   
  

        # result: 79.124.714.123 Pers km


                
        #2
        bus.mileage=(
             bus.transport_capacity_pkm
             /ass("Ass_T_D_lf_ppl_Bus_2050")
         )
         #result: 5.208.314.516 Fz km

       
        #3
        bus.demand_electricity=(
            bus.mileage
            *ass("Ass_T_S_bus_frac_bev_mlg_2050")
            *ass("Ass_T_S_Bus_SEC_elec_2030")
        )
        # result: 8.666.111  MWh/a

        
        #4
        bus.energy=(
            bus.demand_electricity
        )
        # result: 8.666.111  MWh/a
        
        #5 
        bus.CO2e_cb=0 # Ergänzt, um Änderung der CO2e zu berechnen, in Excel leer
        

        #6
        bus.change_energy_MWh=(
             bus.energy
            -root.t18.bus.energy
        )
        #result -459.449 MWh


       
        #7
        bus.change_energy_pct=(
             bus.change_energy_MWh
            /root.t18.bus.energy
        )
        #result: -5%


        #8
        bus.change_CO2e_t=(
             bus.CO2e_cb
            -root.t18.bus.CO2e_cb
        )
        #result: -2.278.190  t/a

        #9
        bus.change_CO2ee_pct=(
            bus.change_CO2e_t
            /root.t18.bus.CO2e_cb
        )
        #result: -100,00%
   
        #10
        bus.CO2e_total_2021_estimated=(
            root.t18.bus.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 2.126.762 t/a



        #11
        bus.cost_climate_saved(
            (bus.CO2e_total_2021_estimated-bus.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 10.160.603.275 €

        
        #12
        bus.change_km=(
            bus.transport_capacity_pkm-root.t18.bus.transport_capacity_pkm
        )
        # 43.529.813.688 Pers km

 
 # Zeile Ausbau Businfrastruktur action_bus_infrastructure
            
        action_bus_infrastructure=root.t3X.action_bus_infrastructure
            
        #1 
        action_bus_infrastructure.action="Ausbau Businfrastruktur"
            
            
        #2
        action_bus_infrastructure.invest_per_x=(
            ass("Ass_T_C_cost_per_trnsprt_ppl_bus_infrstrctr")
        )
        # result: 0,0885 €  
            
        #4 
        action_bus_infrastructure.invest=(
            bus.transport_capacity_pkm
            *action_bus_infrastructure.invest_per_x
        )
        # result: 7.000.000.000 €
            
        #5 
        action_bus_infrastructure.invest_com=(
            action_bus_infrastructure.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 7.000.000.000 €
        
        #6
        action_bus_infrastructure.invest_pa=(
            action_bus_infrastructure.invest
            /entry("In_M_duration_target")
        )
        #result: 777.777.778 €/a
        
        #7
        action_bus_infrastructure.invest_pa_com=(
            action_bus_infrastructure.invest_com
            /entry("In_M_duration_target")
        )
        #result: 777.777.778 €/a  
                        
 # Zeile action bus_invest 
 # Die Spalten zur Maßnahme "Kauf E-busse" aus der Zeile bus sind hier berücksichtigt. 
        #1
        action_bus_invest.action = "Investitionen in Busse mit klimaneutralem Antrieb"
    
        #2
        action_bus_invest.invest_per_x=(
            ass("Ass_T_C_cost_per_trnsprt_ppl_bus")
        )
        # result: 0,1137 € 
        
        #3 Anzahl der Busse im Zieljahr 
        action_bus_invest.base_unit=(
            bus.mileage
             *fact("Fact_T_S_Bus_ratio_mlg_to_stock_2018")
        )
        # result: 110.523 
            
        #4
        action_bus_invest.invest=(
            bus.transport_capacity_pkm
            *action_bus_infrastructure.invest_per_x
        )
        # result: 9.000.000.000 €
            
        #5 
        action_bus_invest.invest_com=(
            action_bus_invest.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 9.000.000.000 €
        
        #6
        action_bus_invest.invest_pa=(
            action_bus_invest.invest
            /entry("In_M_duration_target")
        )
        #result: 1.000.000.000 €/a
        
        #7
        action_bus_invest.invest_pa_com=(
            action_bus_invest.invest_com
            /entry("In_M_duration_target")
        )
        #result: 1.000.000.000 €/a  
            
 #Zeile Güterverkehr LNF (innerorts außerorts) 
 
        ldt_it_ot=root.t3X.ldt_it_ot


       #1
        ldt_it_ot.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_Rd_2050")
            /fact("Fact_T_D_trnsprt_gds_Rd_2018")
            *root.t18.ldt_it_ot.transport_capacity_tkm
        ) 
        #result: 3.384.319.651 t km

        
        #2 
        ldt_it_ot.mileage=(
             ldt_it_ot.transport_capacity_tkm
             /ass("Ass_T_D_lf_gds_LDT_2050")
         )
         # result: 22.546.733.083 Fz km

         #3
        ldt_it_ot.demand_electricity=(
            ldt_it_ot.mileage
            *ass("Ass_T_S_LDT_frac_bev_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_elec_it_at_2030")
        )
        # result: 4.982.731  MWh/a

        #4
        ldt_it_ot.demand_ediesel=(
            ldt_it_ot.mileage
            *ass("Ass_T_S_LDT_frac_diesel_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_diesel_it_at_2030")
        )
        # result: 1.120.503  MWh/a

        #5
        ldt_it_ot.demand_hydrogen=(
            ldt_it_ot.mileage
            *ass("Ass_T_S_LDT_frac_fcev_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_fcev_2030")
        )
        # result: 972.328  MWh/a
        
         #6
        ldt_it_ot.energy=(
            ldt_it_ot.demand_electricity+ldt_it_ot.demand_ediesel+ldt_it_ot.demand_hydrogen
        )
        # result:7.075.563 MWh

        
        #7
        ldt_it_ot.CO2e_cb=(
            ldt_it_ot.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 298.611  t/a
        
        #8 
        ldt_it_ot.change_energy_MWh=(
             ldt_it_ot.energy
            -root.t18.ldt_it_ot.energy
        )
        #result -25.867.708 MWh

       
        #9
        ldt_it_ot.change_energy_pct=(
             ldt_it_ot.change_energy_MWh
            /root.t18.ldt_it_ot.energy
        )
        #result: -79%

        #10
        ldt_it_ot.change_CO2e_t=(
             ldt_it_ot.CO2e_cb
            -root.t18.ldt_it_ot.CO2e_cb
        )
        #result: -7.995.424  t/a


        #11
        ldt_it_ot.change_CO2ee_pct=(
            ldt_it_ot.change_CO2e_t
            /root.t18.ldt_it_ot.CO2e_cb
        )
        #result: -96,40%

        
        #12
        ldt_it_ot.CO2e_total_2021_estimated=(
            root.t18.ldt_it_ot.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 7.742.741 t/a


        #13
        ldt_it_ot.cost_climate_saved(
            (ldt_it_ot.CO2e_total_2021_estimated-ldt_it_ot.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 35.564.328.774 €

        
        #14
        ldt_it_ot.change_km=(
            ldt_it_ot.transport_capacity_tkm-root.t18.ldt_it_ot.transport_capacity_tkm
        )
        # -2.131.794.918 t km


        
 #Zeile Güterverkehr LNF (Autobahn) 
 
        ldt_ab=root.t3X.ldt_ab


       #1
        ldt_ab.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_Rd_2050")
            /fact("Fact_T_D_trnsprt_gds_Rd_2018")
            *root.t18.ldt_ab.transport_capacity_tkm
        ) 
        #result: 1.339.886.307 t km
        
        #2 
        ldt_ab.mileage=(
             ldt_ab.transport_capacity_tkm
             /ass("Ass_T_D_lf_gds_LDT_2050")
         )
         # result: 8.926.479.187 Fz km          

         #3
        ldt_ab.demand_electricity=(
            ldt_ab.mileage
            *ass("Ass_T_S_LDT_frac_bev_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_elec_ab_2030")
        )
        # result: 3.413.541  MWh/a


        #4
        ldt_ab.demand_ediesel=(
            ldt_ab.mileage
            *ass("Ass_T_S_LDT_frac_diesel_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_diesel_ab_2030")
        )
        # result: 617.739  MWh/a


        #5
        ldt_ab.demand_hydrogen=(
            ldt_ab.mileage
            *ass("Ass_T_S_LDT_frac_fcev_mlg_2050")
            *ass("Ass_T_S_LDT_SEC_fcev_2030")
        )
        # result: 384.954  MWh/a
        
        #6
        ldt_ab.energy=(
            ldt_ab.demand_electricity+ldt_ab.demand_ediesel+ldt_ab.demand_hydrogen
        )
        # result:4.416.234 MWh

              
        #7
        ldt_ab.CO2e_cb=(
            ldt_ab.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 164.626  t/a
        
        #8 
        ldt_ab.change_energy_MWh=(
             ldt_ab.energy
            -root.t18.ldt_ab.energy
        )
        #result -13.710.589 MWh


       
        #9
        ldt_ab.change_energy_pct=(
             ldt_ab.change_energy_MWh
            /root.t18.ldt_ab.energy
        )
        #result: -76%

        #10
        ldt_ab.change_CO2e_t=(
             ldt_ab.CO2e_cb
            -root.t18.ldt_ab.CO2e_cb
        )
        #result: -4.397.910  t/a

        #11
        ldt_ab.change_CO2ee_pct=(
            ldt_ab.change_CO2e_t
            /root.t18.ldt_ab.CO2e_cb
        )
        #result: -96,39%
        
        #12
        ldt_ab.CO2e_total_2021_estimated=(
            root.t18.ldt_ab.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 4.259.270 t/a


        #13
        ldt_ab.cost_climate_saved(
            (ldt_ab.CO2e_total_2021_estimated-ldt_ab.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 19.562.161.759 €
        
        #14
        ldt_ab.change_km=(
            ldt_ab.transport_capacity_tkm-root.t18.ldt_ab.transport_capacity_tkm
        )
        # -843.999.124 t km

        
 #Zeile Güterverkehr SNF (innerorts außerorts) 
 
        mhd_it_ot=root.t3X.mhd_it_ot


       #1
        mhd_it_ot.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_Rd_2050")
            /fact("Fact_T_D_trnsprt_gds_Rd_2018")
            *root.t18.mhd_it_ot.transport_capacity_tkm
        ) 
        #result: 130.526.289.921 t km

        #2 
        mhd_it_ot.mileage=(
             mhd_it_ot.transport_capacity_tkm
             /ass("Ass_T_D_lf_gds_MHD_2050")
         )
         # result: 17.443.118.169 Fz km


         #3
        mhd_it_ot.demand_electricity=(
            mhd_it_ot.mileage
            *ass("Ass_T_S_MHD_frac_bev_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_elec_it_ot_2030")
        )
        # result: 8.134.617  MWh/a

        #4
        mhd_it_ot.demand_ediesel=(
            mhd_it_ot.mileage
            *ass("Ass_T_S_MHD_frac_diesel_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_diesel_it_ot_2030")
        )
        # result: 2.604.888  MWh/a


        #5
        mhd_it_ot.demand_hydrogen=(
            mhd_it_ot.mileage
            *ass("Ass_T_S_MHD_frac_fcev_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_fcev_2030")
        )
        # result: 5.007.500  MWh/a
        
        #6
        mhd_it_ot.energy=(
            mhd_it_ot.demand_electricity+mhd_it_ot.demand_ediesel+mhd_it_ot.demand_hydrogen
        )
        # result: 15.747.004 MWh
         
        #7
        mhd_it_ot.CO2e_cb=(
            mhd_it_ot.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 712.776  t/a
        
        #8 
        mhd_it_ot.change_energy_MWh=(
             mhd_it_ot.energy
            -root.t18.mhd_it_ot.energy
        )
        #result -64.685.910 MWh


     
        #9
        mhd_it_ot.change_energy_pct=(
             mhd_it_ot.change_energy_MWh
            /root.t18.mhd_it_ot.energy
        )
        #result: -80%


        #10
        mhd_it_ot.change_CO2e_t=(
             mhd_it_ot.CO2e_cb
            -root.t18.mhd_it_ot.CO2e_cb
        )
        #result: -19.552.247  t/a


        #11
        mhd_it_ot.change_CO2ee_pct=(
            mhd_it_ot.change_CO2e_t
            /root.t18.mhd_it_ot.CO2e_cb
        )
        #result: -96,57%

        
        #12
        mhd_it_ot.CO2e_total_2021_estimated=(
            root.t18.mhd_it_ot.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 18.900.685 t/a


        #13
        mhd_it_ot.cost_climate_saved(
            (mhd_it_ot.CO2e_total_2021_estimated-mhd_it_ot.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 86.981.503.913 €

        
        #14
        mhd_it_ot.change_km=(
            mhd_it_ot.transport_capacity_tkm-root.t18.mhd_it_ot.transport_capacity_tkm
        )
        # result: -82.218.971.690 t km

 #Zeile Güterverkehr SNF (Autobahn) 
 
        mhd_ab=root.t3X.mhd_ab


       #1
        mhd_ab.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_Rd_2050")
            /fact("Fact_T_D_trnsprt_gds_Rd_2018")
            *root.t18.mhd_ab.transport_capacity_tkm
        ) 
        #result: 164.992.675.307 t km

        #2 
        mhd_ab.mileage=(
             mhd_ab.transport_capacity_tkm
             /ass("Ass_T_D_lf_gds_MHD_2050")
         )
         # result: 22.049.096.271 Fz km

         #3
        mhd_ab.demand_electricity=(
            mhd_ab.mileage
            *ass("Ass_T_S_MHD_frac_bev_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_elec_ab_2030")
        )
        # result: 14.251.345  MWh/a

        #4
        mhd_ab.demand_ediesel=(
            mhd_ab.mileage
            *ass("Ass_T_S_MHD_frac_diesel_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_diesel_ab_2030")
        )
        # result: 1.525.863  MWh/a


        #5
        mhd_ab.demand_hydrogen=(
            mhd_ab.mileage
            *ass("Ass_T_S_MHD_frac_fcev_mlg_2050")
            *ass("Ass_T_S_MHD_SEC_fcev_2030")
        )
        # result: 6.329.766  MWh/a

        #6
        mhd_ab.energy=(
            mhd_ab.demand_electricity+mhd_ab.demand_ediesel+mhd_ab.demand_hydrogen
        )
        # result: 22.106.974 MWh

         
        #7
        mhd_ab.CO2e_cb=(
            mhd_ab.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 292.020  t/a
        
        #8 
        mhd_ab.change_energy_MWh=(
             mhd_ab.energy
            -root.t18.mhd_ab.energy
        )
        #result -82.292.165 MWh
   
        #9
        mhd_ab.change_energy_pct=(
             mhd_ab.change_energy_MWh
            /root.t18.mhd_ab.energy
        )
        #result: -79%

        #10
        mhd_ab.change_CO2e_t=(
             mhd_ab.CO2e_cb
            -root.t18.mhd_ab.CO2e_cb
        )
        #result: -25.871.026  t/a


        #11
        mhd_ab.change_CO2ee_pct=(
            mhd_ab.change_CO2e_t
            /root.t18.mhd_ab.CO2e_cb
        )
        #result: -98,45%
        
        #12
        mhd_ab.CO2e_total_2021_estimated=(
            root.t18.mhd_ab.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 24.531.019 t/a


        #13
        mhd_ab.cost_climate_saved(
            (mhd_ab.CO2e_total_2021_estimated-mhd_ab.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 115.254.226.457 €


        #14
        mhd_ab.change_km=(
            mhd_ab.transport_capacity_tkm-root.t18.mhd_it_ot.transport_capacity_tkm
        )
        # result: -103.929.469.751 t km

        
# Zeile Güter (Maßnahmem SNF und LNF) 

        action_mhd_engine_change=root.t3X.action_mhd_engine_change
    
        #1 
        action_mhd_engine_change.action="Antriebswechsel SNF und LNF, Effizienzsteigerungen, Bau Oberleitungsinfrastrukutr "
            
        # 2  
        action_mhd_engine_change.invest=(
            (ass("Ass_T_C_invest_trolley_truck_infrstrctr")
            +ass("Ass_T_C_invest_ldt_engine_change")
            +ass("Ass_T_C_invest_ldt_efficiency"))
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
            
        )
        # result: 157.000.000.000 €
        
    
        #3 
        action_mhd_engine_change.invest_com=(
            ass("Ass_T_C_invest_trolley_truck_infrstrctr")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
        )
        #result: 26.000.000.000 €
        
        #4
        action_mhd_engine_change.invest_pa=(
            action_mhd_engine_change.invest
            /entry("In_M_duration_target")
        )
        #result: 17.444.444.444 €/a
        
        #5
        action_mhd_engine_change.invest_pa_com=(
            action_mhd_engine_change.invest_com
            /entry("In_M_duration_target")
        )
        #result: 2.888.888.889 €/a 
  
 #Zeile Straßenbau  
        action_road_maintenance=root.t3X.action_road_maintenance
    
       #1  
        action_road_maintenance.action= "Erhalt von Straßeninfrastruktur"
        
                # 2  
        action_road_maintenance.invest=(
            ass("Ass_T_C_invest_highways")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
            
        )
        # result: 67.000.000.000 €
        
    
        #3 
        action_road_maintenance.invest_com=(
            action_road_maintenance.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 67.000.000.000 €
        
        #4
        action_road_maintenance.invest_pa=(
            action_road_maintenance.invest
            /entry("In_M_duration_target")
        )
        #result: 7.444.444.444 €/a
        
        #5
        action_road_maintenance.invest_pa_com=(
            action_road_maintenance.invest_com
            /entry("In_M_duration_target")
        )
        #result: 7.444.444.444 €/a    
        


 # Zeile  Ladesäulen 
        action_charge_points=root.t3X.action_charge_points
    
       #1  
        action_charge_points.action= "Aufbau Ladeinfrastruktur"
        
        
        #2  Anzahl öffentlich zugänglicher Ladepunkte  
        if entry("In_T_rt3")=="city": 
                action_charge_points.base_unit=(
                    action_car.base_unit
                    /ass("Ass_S_ratio_bev_car_per_charge_point_city")
            )
    
        elif entry("In_T_rt3")=="smcty":
                action_charge_points.base_unit=(
                    action_car.base_unit
                    /ass("Ass_S_ratio_bev_car_per_charge_point_smcity_rural")
            )         
    
                
        elif entry("In_T_rt3")=="rural":
                action_charge_points.base_unit=(
                    action_car.base_unit
                    /ass("Ass_S_ratio_bev_car_per_charge_point_smcity_rural")
            )
            
        else:
                action_charge_points.base_unit=(
                    action_car.base_unit
                    /ass("Ass_S_ratio_bev_car_per_charge_point_nat")
            ) 
  

        # result: 1.450.000
        
        #3
        action_charge_points_invest_per_x=ass("Ass_T_C_cost_per_charge_point")
        #result: 13.103 € 
        
        
        # 4  
        action_charge_points.invest=(
            action_charge_points_invest_per_x
            *action_charge_points.base_unit      
        )
        # result: 19.000.000.000 €
        
    
        #5
        action_charge_points.invest_com=(
            action_charge_points.invest
            *ass("Ass_T_C_ratio_public_sector_25")
        )
        #result: 4.750.000.000 €
        
        #6
        action_charge_points.invest_pa=(
            action_charge_points.invest
            /entry("In_M_duration_target")
        )
        #result: 2.111.111.111 €/a
        
        #7
        action_charge_points.invest_pa_com=(
            action_charge_points.invest_com
            /entry("In_M_duration_target")
        )
        #result: 527.777.778 €/a   

 # Zeile CRF 1.A.3.b - Straßenverkehr 

        street = root.t3X.street  
        
        # 1
        street.demand_electricity=(
            car_it_ot.demand_electricity
            +car_ab.demand_electricity
            +bus.demand_electricity
            +ldt_it_ot.demand_electricity
            +ldt_ab.demand_electricity
            +mhd_it_ot.demand_electricity
            +mhd_ab.demand_electricity
        )
        #result: 78.742.268  MWh/a
        
        #2 
        street.demand_epetrol=(
            car_it_ot.demand_epetrol
            +car_ab.demand_epetrol
        )
        #result: 6.772.892 MWh
          
        #3
        street.demand_ediesel=(
            +ldt_it_ot.demand_ediesel
            +ldt_ab.demand_ediesel
            +mhd_it_ot.demand_ediesel
            +mhd_ab.demand_ediesel
        )
        # result: 5.868.994 MWh 
            
        #4
        street.demand_hydrogen=(
            +ldt_it_ot.demand_hydrogen
            +ldt_ab.demand_hydrogen
            +mhd_it_ot.demand_hydrogen
            +mhd_ab.demand_hydrogen
        )
        # result: 12.694.548 MWh    
        
        #5
        street.energy=(
            car_it_ot.energy
            +car_ab.energy
            +bus.energy
            +ldt_it_ot.energy
            +ldt_ab.energy
            +mhd_it_ot.energy
            +mhd_ab.energy
        )
        #result: 104.078.701 MWh
              
        #6
        street.CO2e_cb=(
            car_it_ot.CO2e_cb
            +car_ab.CO2e_cb
            +ldt_it_ot.CO2e_cb
            +ldt_ab.CO2e_cb
            +mhd_it_ot.CO2e_cb
            +mhd_ab.CO2e_cb
        )
        #result: 3.399.725  t/a
        
        #6 
        street.change_energy=(
            street.energy
            -root.t18.street.energy
        )
        #result: -586.914.804 MWh
        
        #7 
        street.change_energy_pct=(
            street.change_energy_MWh
            /root.t18.street.energy
        )
        #result: -85%

        #8
        street.change_CO2e_t=(
            street.CO2e_cb
            -root.t18.street.CO2e_cb
        )
        #result: -172.039.673  t/a

        #9
        street.change_CO2ee_pct=(
            street.change_CO2e_t
            /root.t18.street.CO2e_cb
        )
        #result: -98,06%

            
        #10
        streeet.CO2e_total_2021_estimated=(
            car_it_ot.CO2e_total_2021_estimated
            +car_ab.CO2e_total_2021_estimated
            +bus.CO2e_total_2021_estimated
            +ldt_it_ot.CO2e_total_2021_estimated
            +ldt_ab.CO2e_total_2021_estimated
            +mhd_it_ot.CO2e_total_2021_estimated
            +mhd_ab.CO2e_total_2021_estimated       
        )
        #result: 163.778.148 t/a


        #11
        street.cost_climate_saved=(
            car_it_ot.cost_climate_saved
            +car_ab.cost_climate_saved
            +bus.cost_climate_saved
            +ldt_it_ot.cost_climate_saved
            +ldt_ab.cost_climate_saved
            +mhd_it_ot.cost_climate_saved
            +mhd_ab.cost_climate_saved      
        )
        # result: 766.207.915.728 €

        #12 
        street.invest=(
            action_car.invest
            +action_bus_invest.invest
            +action_bus_infrastructure.invest
            +action_mhd_engine_change.invest
            +action_road_maintenance.invest
            +action_charge_points.invest    
        )
        # result: 1.014.135.142.857 €
        
        #13
        street.invest_com=(
            action_car.invest_com
            +action_bus_invest.invest_com
            +action_bus_infrastructure.invest_com
            +action_mhd_engine_change.invest_com
            +action_road_maintenance.invest_com
            +action_charge_points.invest_com    
        )
        #result: 113.750.000.000 €
        
        #14
        street.invest_pa=(
            action_car.invest_pa
            +action_bus_invest.invest_pa
            +action_bus_infrastructure.invest_pa
            +action_mhd_engine_change.invest_pa
            +action_road_maintenance.invest_pa
            +action_charge_points.invest_pa   
        )
        
        # result: 112.681.682.540 €
        
        #15
        street.invest_pa_com=(
            action_car.invest_pa_com
            +action_bus_invest.invest_pa_com
            +action_bus_infrastructure.invest_pa_com
            +action_mhd_engine_change.invest_pa_com
            +action_road_maintenance.invest_pa_com
            +action_charge_points.invest_pa_com  
        )
        
        # result: 12.638.888.889 €
        
        
        
 #Zeile Eisenbahnverkehr Personen

 
        rail_ppl=root.t3X.rail_ppl

       #1
        if entry("In_T_rt3")=="city": 
            rail_ppl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.rail_ppl.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_city_pt_frac_2050")

            )
    
    
    
        elif entry("In_T_rt3")=="smcty":
            rail_ppl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.rail_ppl.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_smcty_pt_frac_2050")

            )            
    
                
        elif entry("In_T_rt3")=="rural":
            rail_ppl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.rail_ppl.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_rural_pt_frac_2050")
            )   
            
        else:
            rail_ppl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.rail_ppl.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_nat_pt_frac_2050")
                )   
  

        # result: 217.179.553.121 Pers km
                
        #2
        rail_ppl.demand_electricity=(
            rail_ppl.mileage
            *ass("Ass_T_S_Rl_Train_ppl_long_elec_SEC_2050")
        )
        # result: 15.739.419  MWh/a
        
        
        #3 
        rail_ppl.energy=rail_ppl.demand_electricity
        # result: 15.739.419  MWh/a       
        
        #4 
        rail_ppl.CO2e_cb=0 # Ergänzt, um Änderung der CO2e zu berechnen, in Excel leer
        
        #5 
        rail_ppl.change_energy_MWh=(
             rail_ppl.energy
            -root.t18.rail_ppl.energy
        )
        #result 6.092.821 MWh

   
        #6
        rail_ppl.change_energy_pct=(
             rail_ppl.change_energy_MWh
            /root.t18.rail_ppl.energy
        )
        #result: 63%


        #7
        rail_ppl.change_CO2e_t=(
             rail_ppl.CO2e_cb
            -root.t18.rail_ppl.CO2e_cb
        )
        #result: -843.358  t/a



        #8
        rail_ppl.change_CO2ee_pct=(
            rail_ppl.change_CO2e_t
            /root.t18.rail_ppl.CO2e_cb
        )
        #result: -100 %
        
        #9
        rail_ppl.CO2e_total_2021_estimated=(
            root.t18.rail_ppl.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 787.301 t/a


        #10
        rail_ppl.cost_climate_saved(
            (rail_ppl.CO2e_total_2021_estimated-rail_ppl.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 3.761.332.169 €


        #11
        rail_ppl.change_km=(
            rail_ppl.transport_capacity_pkm-root.t18.rail_ppl.transport_capacity_pkm
        )
        # result: 119.479.553.121 Pers km

 
 #Zeile SSU-Bahn (im ÖPNV)

        metro=root.t3X.metro

       #1
        if entry("In_T_rt3")=="city": 
            metro.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.metro.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_city_pt_frac_2050")

            )
    
    
    
        elif entry("In_T_rt3")=="smcty":
            metro.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.metro.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_smcty_pt_frac_2050")

            )            
    
                
        elif entry("In_T_rt3")=="rural":
            metro.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.metro.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_rural_pt_frac_2050")
            )   
            
        else:
            metro.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *root.t18.metro.transport_capacity_pkm
                /(root.t18.bus.transport_capacity_pkm
                  +root.t18.rail_ppl.transport_capacity_pkm
                  +root.t18.metro.transport_capacity_pkm
                 )
                *ass("Ass_T_D_trnsprt_ppl_nat_pt_frac_2050")
                )   
  

        # result: 39.345.732.756 Pers km
               
        #2
        metro.mileage=(
             metro.transport_capacity_pkm
             /ass("Ass_T_D_lf_Rl_Metro_2050")
         )
         #result: 686.660.839 Fz km

       
        #3
        metro.demand_electricity=(
            metro.mileage
            *ass("Ass_T_S_Rl_Metro_SEC_fzkm_2050")
        )
        # result: 2.746.643  MWh/a
        
        #4
        metro.energy=metro.demand_electricity
        # result: 2.746.643  MWh/a   
        

        
        #5 
        metro.change_energy_MWh=(
            metro.energy
            -root.t18.metro.energy
        )
        #result 1.511.043 MWh


        #6
        metro.change_energy_pct=(
             metro.change_energy_MWh
            /root.t18.metro.energy
        )
        #result: 122%

        #7
        metro.change_km=(
            metro.transport_capacity_pkm-root.t18.metro.transport_capacity_pkm
        )
        # result: 21.645.732.756 Pers km
        
        #8 
        metro.action="Investitionen in SSU-Bahn (Infrastruktur und Fahrzeuge)"
        # Todo: Prüfen 
        
        #9 
        metro.invest_per_x=(
            ass("Ass_T_C_cost_per_trnsprt_ppl_metro")
        )        
        #result: 0,762 €/Pkm
        
        #10
        metro.invest=(
            metro.transport_capacity_pkm
            *metro.invest_per_x
        )
        #result: 30.000.000.000 €
        
        #11
        metro.invest_com=(
            metro.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 30.000.000.000 €
        
        #12
        metro.invest_pa=(
            metro.invest
            /entry("In_M_duration_target")
        )
        #result: 3.333.333.333 €/a
        
        #13
        metro.invest_pa_com=(
            metro.invest_com
            /entry("In_M_duration_target")
        )
        #result: 3.333.333.333 €/a
        
        
        

  #Zeile Güterverkehr Schiene

        rail_gds=root.t3X.rail_gds

       #1
        rail_gds.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_Rl_2050")
            /fact("Fact_T_D_Rl_train_nat_trnsprt_gds_2018")
            *root.t18.rail_gds.transport_capacity_tkm
        ) 
        #result: 202.000.000.000 t km
        
        #2
        rail_gds.demand_electricity=(
            rail_ppl.mileage
            *ass("Ass_T_S_Rl_Train_gds_elec_SEC_2050")
        )
        # result: 6.302.522  MWh/a
        
        #3
        rail_gds.energy=rail_gds.demand_electricity
        # result: 6.302.522 MWh

        #4 
        rail_gds.CO2e_cb=0 # Ergänzt, um Änderung der CO2e zu berechnen, in Excel leer
        
        #5 
        rail_gds.change_energy_MWh=(
             rail_gds.energy
            -root.t18.rail_gds.energy
        )
        #result 1.233.639 MWh


   
        #6
        rail_gds.change_energy_pct=(
             rail_gds.change_energy_MWh
            /root.t18.rail_gds.energy
        )
        #result: 24%



        #7
        rail_gds.change_CO2e_t=(
             rail_gds.CO2e_cb
            -root.t18.rail_gds.CO2e_cb
        )
        #result: -265.279  t/a

        #8
        rail_gds.change_CO2ee_pct=(
            rail_gds.change_CO2e_t
            /root.t18.rail_gds.CO2e_cb
        )
        #result: -100 %
        
        #9
        rail_gds.CO2e_total_2021_estimated=(
            root.t18.rail_gds.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 247.646 t/a



        #10
        rail_gds.cost_climate_saved(
            (rail_gds.CO2e_total_2021_estimated-rail_gds.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 1.183.131.087 €


        #11
        rail_gds.change_km=(
            rail_gds.transport_capacity_tkm-root.t18.rail_gds.transport_capacity_tkm
        )
        # result: 68.300.000.000 t km
        
        
# Zeile Personen + Güter (BVWP (Erhalt) + Aufbau Schiene (Güter und Personen))
        action_rail_maintenance=root.t3X.action_rail_maintenance
    
       #1  
        action_rail_maintenance.action= "Erhalt und Ausbau von Schieneninfrastruktur"
        
        # 2  
        action_rail_maintenance.invest=(
            (ass("Ass_T_C_additional_invest_train_net_tech")
            +ass("Ass_T_C_invest_rail_ways"))
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
            
        )
        # result: 332.400.000.000 €
        
    
        #3 
        action_rail_maintenance.invest_com=(
            action_rail_maintenance.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 332.400.000.000 €
        
        #4
        action_rail_maintenance.invest_pa=(
           action_rail_maintenance.invest
            /entry("In_M_duration_target")
        )
        #result: 36.933.333.333 €/a
        
        #5
        action_rail_maintenance.invest_pa_com=(
            action_rail_maintenance.invest_com
            /entry("In_M_duration_target")
        )
        #result: 36.933.333.333 €/a   
        




# Zeile Schienenverkehr Invest (Schienenverkehr Investitionen in Züge und Bahnhöfe)

        action_rail_invest=root.t3X.action_rail_invest
    
       #1  
        action_rail_invest.action= "Erhalt und Ausbau von Schieneninfrastruktur"
        
        # 2  
        action_rail_invest.invest=(
            (ass("Ass_T_C_additional_invest_train")
            +ass("Ass_T_C_additional_invest_train_station"))
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
            
        )
        # result: 55.000.000.000 €
        
    
        #3 
        action_rail_invest.invest_pa=(
           action_rail_maintenance.invest
            /entry("In_M_duration_target")
        )
        #result: 6.111.111.111 €/a
        
# Zeile CRF 1.A.3.c - Schienenverkehr

        rail = root.t3X.rail 
        
        # 1
        rail.demand_electricity=(
            rail_ppl.demand_electricity
            +rail_gds.demand_electricity
            +metro.demand_electricity
        )
        #result: 24.788.584  MWh/a  
        
        #2
        rail.energy=(
            rail_ppl.energy
            +rail_gds.energy
            +metro.energy
        )
        #result: 24.788.584 MWh
       
        rail.CO2e_cb =  rail_ppl.CO2e_cb+ rail_gds.CO2e_cb  # result: 0 ergänzt für Veränderung 2018-203X

        #3 
        rail.change_energy=(
            rail.energy
            -root.t18.rail.energy
        )
        #result: 8.837.503 MWh
        
        #4
        rail.change_energy_pct=(
            rail.change_energy_MWh
            /root.t18.rail.energy
        )
        #result: 55%

        #5
        rail.change_CO2e_t=(
            rail.CO2e_cb
            -root.t18.rail.CO2e_cb
        )
        #result: -1.108.638  t/a

        #6
        rail.change_CO2ee_pct=(
            rail.change_CO2e_t
            /root.t18.rail.CO2e_cb
        )
        #result: -100,00%

            
        #7
        rail.CO2e_total_2021_estimated=(
            rail_ppl.CO2e_total_2021_estimated
            +rail_gds.CO2e_total_2021_estimated 
        )
        #result: 1.034.948 t/a


        #8
        rail.cost_climate_saved=(
            rail_ppl.cost_climate_saved
            +rail_gds.cost_climate_saved   
        )
        # result: 4.944.463.256 €

        #9
        rail.invest=(
            metro.invest
            +action_rail_maintenance.invest
            +action_rail_invest.invest
        )
        # result: 417.400.000.000 €
        
        #10
        rail.invest_com=(
            metro.invest_com
            +action_rail_maintenance.invest_com
        )
        #result: 113.750.000.000 €
        
        #11
        rail.invest_pa=(
            metro.invest_pa
            +action_rail_maintenance.invest_pa
            +action_rail_invest.invest_pa
        )
        
        # result: 46.377.777.778 €
        
        #12
        rail.invest_pa_com=(
            metro.invest_pa_com
            +action_rail_maintenance.invest_pa_com
        )
        
        # result: 40.266.666.667 €


#Zeile Güterverkehr Binnenschifffahrt

        ship_dmstc=root.t3X.ship_dmstc

       #1
        ship_dmstc.transport_capacity_tkm=(
            ass("Ass_T_D_trnsprt_gds_ship_2050")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")  
        ) 
        #result: 71.000.000.000 t km
        
        #2
        ship_dmstc.demand_ediesel=(    
            ass("Ass_T_D_Shp_dmstc_nat_EB_2050")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")  
        )
        # result: 3.000.000 MWh
        
        #3
        ship_dmstc.energy= ship_dmstc.demand_ediesel
        # result: 3.000.000 MWh
        
        #4
        ship_dmstc.CO2e_cb=(
            ship_dmstc.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 799.492  t/a


        #5 
        ship_dmstc.change_energy_MWh=(
             ship_dmstc.energy
            -root.t18.ship_dmstc.energy
        )
        #result 50.278 MWh


        #6
        ship_dmstc.change_energy_pct=(
             ship_dmstc.change_energy_MWh
            /root.t18.ship_dmstc.energy
        )
        #result: 2%


        #7
        ship_dmstc.change_CO2e_t=(
             ship_dmstc.CO2e_cb
            -root.t18.ship_dmstc.CO2e_cb
        )
        #result: 13.399  t/a


        #8
        ship_dmstc.change_CO2ee_pct=(
            ship_dmstc.change_CO2e_t
            /root.t18.ship_dmstc.CO2e_cb
        )
        #result: 1,70%

        
        #9
        ship_dmstc.CO2e_total_2021_estimated=(
            root.t18.ship_dmstc.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 733.842 t/a


        #10
        ship_dmstc.cost_climate_saved(
            (ship_dmstc.CO2e_total_2021_estimated-ship_dmstc.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: -313.640.694 €


        #11
        ship_dmstc.change_km=(
            ship_dmstc.transport_capacity_tkm-root.t18.ship_dmstc.transport_capacity_tkm
        )
        # result: 24.100.000.000 t km

        
        # 12 
        ship_dmstc.action=("Maßnahmen zum Erhalt und Ausbau der Bundeswasserstraßen")
        
        #13 
        ship_dmstc.invest=(
            ass("Ass_T_C_invest_water_ways")
            *entry("In_M_population_com_2018")
            /entry("In_M_population_com_203X")
        )
        #result: 24.500.000.000 €
        
        #14
        ship_dmstc.invest_com=(
            ship_dmstc.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 24.500.000.000 €
        
        #15
        ship_dmstc.invest_pa=(
            ship_dmstc.invest
            /entry("In_M_duration_target")
        )
        #result: 2.722.222.222 €/a
        
        #16
        ship_dmstc.invest_pa_com=(
            ship_dmstc.invest_com
            /entry("In_M_duration_target")
        )
        #result: 2.722.222.222 €/a
             

#Zeile Seeschifffahrt

        ship_inter=root.t3X.ship_inter

       #1
        ship_inter.demand_ediesel=(    
            ass("Ass_T_D_Shp_sea_nat_EB_2050")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")  
        )
        # result: 10.000.000 MWh

       #2
        ship_inter.energy= ship_inter.demand_ediesel
        # result: 10.000.000 MWh
         
     
    
        #3
        ship_inter.CO2e_cb=(
            ship_dmstc.demand_ediesel
            *ass("Ass_T_S_diesel_EmFa_tank_wheel_2050")
        )
        # result: 2.664.972  t/a
        
        #4
        ship_inter.change_energy_MWh=(
             ship_inter.energy
            -root.t18.ship_inter.energy
        )
        #result -9.722.222 MWh

        #5
        ship_inter.change_energy_pct=(
             ship_inter.change_energy_MWh
            /root.t18.ship_inter.energy
        )
        #result: -49%

        #6
        ship_inter.change_CO2e_t=(
             ship_inter.CO2e_cb
            -root.t18.ship_inter.CO2e_cb
        )
        #result: -2.731.028  t/a


        #7
        ship_inter.change_CO2ee_pct=(
            ship_inter.change_CO2e_t
            /root.t18.ship_inter.CO2e_cb
        )
        #result: -50,61%


        #8
        ship_inter.CO2e_total_2021_estimated=(
            root.t18.ship_inter.CO2e_cb
            *fact("Fact_M_CO2e_wo_lulucf_2021_vs_2018")
        )
        #result: 5.037.334 t/a

        #9
        ship_inter.cost_climate_saved(
            (ship_inter.CO2e_total_2021_estimated-ship_inter.CO2e_cb)
            *entry("In_M_duration_neutral")
            *fact("Fact_M_cost_per_CO2e_2020")
        )
        # result: 11.333.960.587 €
        
        
# Zeile CRF 1.A.3.d - Küsten- & Binnenschifffahrt

        ship = root.t3X.ship 
        
        # 1
        ship.demand_ediesel=(
            ship_dmstc.demand_ediesel
            +ship_inter.demand_ediesel
        )
        #result: 13.000.000 MWh  
        
        #2
        ship.energy=(
            ship_dmstc.energy
            +ship_inter.energy
        )
        #result: 13.000.000 MWh 
              
        #3 
        ship.CO2e_cb=(
            ship_dmstc.CO2e_cb
            +ship_inter.CO2e_cb
        )
        # result: 3.464.464  t/a
        
        
        #4
        ship.change_energy=(
            ship.energy
            -root.t18.ship.energy
        )
        #result: -9.671.944 MWh
        
        #5
        ship.change_energy_pct=(
            ship.change_energy_MWh
            /root.t18.ship.energy
        )
        #result: -43%

        #6
        ship.change_CO2e_t=(
            ship.CO2e_cb
            -root.t18.ship.CO2e_cb
        )
        #result: -2.717.629  t/a

        #7
        ship.change_CO2ee_pct=(
            ship.change_CO2e_t
            /root.t18.ship.CO2e_cb
        )
        #result: -43,96%

            
        #8
        ship.CO2e_total_2021_estimated=(
            ship_dmstc.CO2e_total_2021_estimated
            +ship_inter.CO2e_total_2021_estimated 
        )
        #result: 5.771.176 t/a


        #9
        ship.cost_climate_saved=(
            ship_dmstc.cost_climate_saved
            +ship_inter.cost_climate_saved   
        )
        # result: 11.020.319.892 €

        #10
        ship.invest=(
            ship_dmstc.invest
        )
        # result: 24.500.000.000 €
        
        #11
        ship.invest_com=(
            ship_dmstc.invest_com
        )
        #result: 24.500.000.000 €
        
        #12
        ship.invest_pa=(
            ship_dmstc.invest_pa
        )     
        # result: 2.722.222.222 €
        
        #13
        ship.invest_pa_com=(
            ship_dmstc.invest_pa_com
        ) 
        # result: 2.722.222.222 €

 #Zeile Fußverkehr

        foot=root.t3X.foot


       #1
        if entry("In_T_rt3")=="city": 
            foot.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_city_foot_frac_2050")
            )
    
    
    
        elif entry("In_T_rt3")=="smcty":
            foot.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_smcty_foot_frac_2050")
            )            
    
                
        elif entry("In_T_rt3")=="rural":
            foot.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_rural_foot_frac_2050")
            )   
            
        else:
            foot.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_nat_foot_frac_2050")
                )   
  

        # result: 38.360.000.000 Pers km
    
        #2
        foot.change_km=(
            foot.transport_capacity_pkm-root.t18.foot.transport_capacity_pkm
        )
        # result: 8.057.987.255 Pers km

    
      
 #Zeile Radverkehr

        cycl=root.t3X.cycl

       #1
        if entry("In_T_rt3")=="city": 
            cycl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_city_cycl_frac_2050")
            )
    
    
    
        elif entry("In_T_rt3")=="smcty":
            cycl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_smcty_cycl_frac_2050")
            )            
    
                
        elif entry("In_T_rt3")=="rural":
            cycl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_rural_cycl_frac_2050")
            )   
            
        else:
            cycl.transport_capacity_pkm=(
                T.transport_capacity_pkm
                *ass("Ass_T_D_trnsprt_ppl_nat_cycl_frac_2050")
                )   
  

        # result: 57.540.000.000 Pers km
                 
        #2
        cycl.change_km=(
            cycl.transport_capacity_pkm-root.t18.cycl.transport_capacity_pkm
        )
        # result:27.237.987.255 Pers km
        
        #3 
        cycl.action=("Investitionen in Rad- und Fußverkehr, sowie Bike&Ride Stationen")
        # Prüfen
        
        #4 
        cycl.invest_per_x=(
            ass("Ass_T_C_cost_per_trnsprt_ppl_cycle")
        )        
        #result:  0,0869 € 
        
        #10
        cycl.invest=(
           cycl.transport_capacity_pkm
            *cycl.invest_per_x
        )
        #result: 5.000.000.000 €
        
        #11
        cycl.invest_com=(
            cycl.invest
            *ass("Ass_T_C_ratio_public_sector_100")
        )
        #result: 5.000.000.000 €
        
        #12
        cycl.invest_pa=(
            cycl.invest
            /entry("In_M_duration_target")
        )
        #result: 555.555.556 €/a
        
        #13
        cycl.invest_pa_com=(
            cycl.invest_com
            /entry("In_M_duration_target")
        )
        #result: 555.555.556 €/a
        
 # Zeile Sonstige nicht CRF
        others=root.t3X.others
       
        #1
        others.invest=(
            cycl.invest
        )
        # result: 5.000.000.000 €
        
        #2
        others.invest_com=(
            cycl.invest_com
        )
        #result: 5.000.000.000 €
        
        #3
        others.invest_pa=(
            cycl.invest_pa
        )     
        # result: 555.555.556 €
        
        #4
        others.invest_pa_com=(
            cycl.invest_pa_com
        ) 
        # result: 555.555.556 €
 
 # Zeile Planung
        planning=root.t3X.planning
      
         #1 
        planning.cost_wage=(
            ass("Ass_T_C_planer_cost_per_invest_cost")
            *(action_bus_infrastructure.invest_pa
              +action_road_maintenance.invest_pa
              +rail_metro.invest_pa
              +cycl.invest_pa
             )
        )
        # result 1.534.666.667 €/a
    
        #2
        plannning.ratio_wage_to_emplo=ass("Ass_T_C_yearly_costs_per_planer")
        # result: 100.000 €/a 
    
        #3
        planning.demand_emplo = (
            planning.cost_wage/plannning.ratio_wage_to_emplo
        )
        # result: 15.347
        
 # Zeile T (Fortsetzung)

        #2
        T.demand_electricity=(
            street.demand_electricity
            +rail.demand_electricity
        )
        #result: 103.530.852  MWh/a
        
        
        #3
        T.demand_epetrol=(
            street.demand_epetrol
        )
        #result: 6.772.892 MWh
        
        #4
        T.demand_ediesel=(
            street.demand_ediesel
            +ship.demand_ediesel
        )
        #result: 18.868.994 MWh
        
        #5 
        T.demand_epetrol=(
            air.demand_epetrol
        )
        #result: 42.000.000 MWh
        
        #6 
        T.demand_hydrogen=(
            street.demand_hydrogen
        )
        #result: 12.694.548 MWh
    
        #7
        T.energy=(
            air.energy
            +street.energy
            +rail.energy
            +ship.energy      
        )
        #result: 183.867.285 MWh
        
        #8
        T.CO2e_cb=(
            air.CO2e_cb
            +street.CO2e_cb
            +ship.CO2e_cb     
        )
        #result: 17.940.495  t/a
        
        #9
        T.CO2e_total=(
            T.CO2e_cb
        )
        #result: 17.940.495  t/a 
        
        #10
        T.change_energy=(
            T.energy
            -root.t18.T.energy
        )
        #result: -667.385.078 MWh
        
        #11
        T.change_energy_pct=(
            T.change_energy_MWh
            /root.t18.T.energy
        )
        #result: -78%

        #12
        T.change_CO2e_t=(
            T.CO2e_cb
            -root.t18.T.CO2e_cb
        )
        #result: -196.895.649  t/a

        #13
        T.change_CO2ee_pct=(
            T.change_CO2e_t
            /root.t18.T.CO2e_cb
        )
        #result: -91,65%

            
        #14
        T.CO2e_total_2021_estimated=(
            air.CO2e_total_2021_estimated
            +street.CO2e_total_2021_estimated
            +rail.CO2e_total_2021_estimated
            +ship.CO2e_total_2021_estimated
        )
        #result: 200.556.239 t/a

        #15
        T.cost_climate_saved=(
            air.cost_climate_saved
            +street.cost_climate_saved
            +rail.cost_climate_saved
            +ship.cost_climate_saved
        )
        # result: 872.446.714.415 €
         
        #16 
        T.invest=(
            +street.invest
            +rail.invest
            +ship.invest
            +others.invest
        )
        # result: 1.461.035.142.857 €
        
        #17 
        T.invest_com=(
            +street.invest_com
            +rail.invest_com
            +ship.invest_com
            +others.invest_com
        )
        # result: 505.650.000.000 €
        
        #18
        T.invest_com=(
            +street.invest_pa
            +rail.invest_pa
            +ship.invest_pa
            +others.invest_pa
        )
        # result: 162.337.238.095 €
        
        #19 
        T.invest_pa_com=(
            +street.invest_pa_com
            +rail.invest_pa_com
            +ship.invest_pa_com
            +others.invest_pa_com
        )
        # result: 56.183.333.333 €

        #20
        T.emplo_existing=(   
            (fact("Fact_T_constr_roads_railway_emplo_2018")+fact("Fact_T_land_trnsp_services_emplo_2018"))  
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
        )
        # result: 1.929.195 ppl 
        
        #21
        T.demand_emplo_new=(
            ass("Ass_T_D_nat_employees")
            *entry("In_M_population_com_203X")
            /entry("In_M_population_nat")
        )
        #result: 740.000 ppl
 
        #22
        T.demand_emplo=(
            T.emplo_existing
            +T.demand_emplo_new
        )
        #result: 2.669.195 ppl 

# Zeilen Bereitstellung 

    root.t3X.s_epetrol= T.demand_epetrol #6.772.892 MWh
    root.t3X.s_ejetfuel = T.demand_ejetfuel #42.000.000 MWh
    root.t3X.s_ediesel = T.demand_ediesel #18.868.994 MWh
    root.t3X.s_elec = T.demand_electricity #103.530.852 MWh
    root.t3X.s_hydrogen = T.demand_hydrogen  #12.694.548 MWh

    
   except Exception as e:
    print(e)
